In [6]:
import pandas as pd
import numpy as np
import json
from datetime import date, timedelta, datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import urllib.request
from selenium.webdriver.common.keys import Keys
import os

In [7]:
date_2year_ago = (date.today() - timedelta(days=2*365))
date_today = date.today()
print('Las fechas a utilizar son: fecha dos anios atras: ' + str(date_2year_ago) + ' y fecha actual: ' + str(date_today))
# Cambiar esta variable si se necesita una fecha especifica
date_2year_ago = 20180000#int(date_2year_ago.strftime('%Y%m%d'))
date_today = int(date_today.strftime('%Y%m%d'))

Las fechas a utilizar son: fecha dos anios atras: 2019-07-23 y fecha actual: 2021-07-22


In [3]:
def first_df():
    # leemos desde la URL el json que contiene la tabla de informacion acerca de las tasas
    df = pd.read_json('https://www.datos.gov.co/resource/mcec-87by.json')
    # hacemos transformaciones a las fechas para tenerlas YYYYMMDD numericas
    df['vigenciadesde'] = pd.to_datetime(df['vigenciadesde']).dt.strftime('%Y%m%d').astype('int32')
    df['vigenciahasta'] = pd.to_datetime(df['vigenciahasta']).dt.strftime('%Y%m%d').astype('int32')
    # retornamos el data frame desde la fecha 2019
    return df[df['vigenciadesde'] >= date_2year_ago]

In [4]:
df  = first_df()
df

,valor,unidad,vigenciadesde,vigenciahasta
0,3796.07,COP,20210715,20210715
1,3826.85,COP,20210714,20210714
2,3824.08,COP,20210713,20210713
3,3829.46,COP,20210710,20210712
4,3850.46,COP,20210709,20210709
...,...,...,...,...
835,2914.37,COP,20180110,20180110
836,2898.32,COP,20180106,20180109
837,2885.76,COP,20180105,20180105
838,2908.68,COP,20180104,20180104


In [5]:
# Se agrupa por anio y mes. Se cuenta para saber cuantos registros hay por cada mes lo que equivale
# a la cantidad de dias por mes
tasa_spot = first_df()
periodo = (tasa_spot['vigenciadesde']/100).astype('int32')
data = periodo.to_list()
names = ['periodo']
new_df = pd.DataFrame(data, columns=names)
new_df.value_counts().sort_index()

periodo
201801     19
201802     19
201803     20
201804     20
201805     20
201806     20
201807     18
201808     21
201809     20
201810     20
201811     19
201812     20
201901     19
201902     19
201903     21
201904     19
201905     21
201906     19
201907     20
201908     21
201909     19
201910     22
201911     19
201912     20
202001     19
202002     20
202003     20
202004     20
202005     20
202006     18
202007     22
202008     19
202009     21
202010     22
202011     16
202012     21
202101     18
202102     19
202103     21
202104     20
202105     20
202106     19
202107     10
dtype: int64

In [9]:
# Segundo dataframe
# Se configuran las opciones del navegador para que empiece maximixado y sin extensiones habilitadas
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

In [10]:
# Funcion para eliminar duplicados de una lista
def remove_duplicate_list(lst):
    ''' Funcion para eliminar duplicados de una lista'''
    return list(dict.fromkeys(lst))

# Se crea instancia del driver que abrira el navegador web con las opciones definidas anteriormente
# Cambiar direccion a donde se encuentra el driver
driver_path = r'C:\Users\Efra\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path, options = options)
# Se le pasa el link a abrir en el navegador
driver.get('https://totoro.banrep.gov.co/analytics/saw.dll?Portal&PortalPath=%2Fshared%2FDashboards_T%2FD_Estad%C3%ADsticas%2FEstad%C3%ADsticas&NQUser=publico&NQPassword=publico123&lang=es&page=Tasas%20de%20inter%C3%A9s%20y%20sector%20financiero&pagina=Otras%20tasas%20de%20inter%C3%A9s')

WebDriverWait(driver, 15)
driver.switch_to.frame(driver.find_element_by_id('frame_dashboard'))

# Se da click en el boton otras tasas de interes
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.XPATH,
                                      '/html/body/div[7]/div/div/table/tbody/tr/td[1]/div/div/table/tbody/tr/td[2]/div/table/tbody/tr/td[2]/table/tbody/tr/td/div')))\
    .click()

# Se da click en el menu desplegable Indicador bancario de referencia IBR
WebDriverWait(driver, 15)\
.until(EC.element_to_be_clickable((By.CLASS_NAME,
                                      'sui-treeview-item-toggle-collapsed')))
driver.find_elements_by_class_name('sui-treeview-item-toggle-collapsed')[1].click()


#print(len(driver.find_elements_by_xpath("//a[@href]")))
# Se guardan todos los links asocaidos al atributo o etiqueta href
href = driver.find_elements_by_xpath("//a[@href]")
href_list = []
# Se recorre la lista href en busca de los links que pertenezcan a los insumos de IBR
for i in href:
    try:
        contains = str(i.get_attribute('href')).replace('%20', '').upper()
        #print(contains)
        # Si el href contiene alguno de los strings, es guardado en una lista que contendra los enlaces a descargar
        if('IBR_PLAZOOVERNIGHTNOMINALPARAUNRANGODEFECHASDADOIQY' in contains 
           or 'IBR_PLAZOUNMESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOTRESMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOMESVENCIDONOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           or 'IBR_PLAZOSEISMESESNOMINALPARAUNRANGODEFECHASDADOIQY' in contains
           and 'DOWNLOAD' in contains):
            src = i.get_attribute('href')
            href_list.append(src)
            #print('saca algo ' + src)
    except:
        #print(error)
        print('error href')

# Como la lista posiblemente guarde enlaces duplicados, se aplica la funcion para remover duplicados y asi descargar una sola vez
# cada archivo
href_list = remove_duplicate_list(href_list)
print('Cantidad de archivos a descargar: ' + str(len(href_list)))
for j in href_list:
    try:
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 't') 
        driver.get(j)
        print('Archivo descargado asociado a: ' + j)
    except AssertionError as error:
        print(error)
        print('error link')
        
# Cambiar a 60 segundos para que descargue antes de cerrar
time.sleep(30)
driver.close()

error href
error href
error href
error href
error href
Cantidad de archivos a descargar: 5
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.1.IBR_Plazo%20overnight%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.2.IBR_Plazo%20un%20mes%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://totoro.banrep.gov.co/analytics/saw.dll?Download&Format=excel2007&Extension=.xls&BypassCache=true&lang=es&path=%2Fshared%2FSeries%20Estad%C3%ADsticas_T%2F1.%20IBR%2F%201.3.IBR_Plazo%20tres%20meses%20nominal%20para%20un%20rango%20de%20fechas%20dado%20IQY
Archivo descargado asociado a: https://t

# Tratamiento archivos IBR

In [8]:
def clean_ibr_df(ruta):
    '''Funcion que limpia y adecua los archivos IBR, como parametro recibe la ruta del archivo excel a tratar'''
    # Limpieza y adecuacion primer data frame IBR plazo overnight
    # Se lee la ruta donde se encuentra el archivo excel IBR y se le indica que los encabezados estaran en la posicion o celda 8
    # y que omita los pie de pagina
    df = pd.read_excel(ruta, header = 8, skipfooter = 7)
    # se renombran las columnas para mas claridad en el df
    df.rename(columns = {'Fecha (dd/mm/aaaa)' : 'Fecha', 'IBR' : 'IBR_efectivo', 'IBR.1' : 'IBR_nominal'}, inplace = True)
    # Se valida si la columna plazo existe, si no existe automaticamente le inserta una nueva columna al df y la rellena con el valor 0
    if 'Plazo real (días calendario)' not in df.columns:
        df['Plazo real (días calendario)'] = 0
    # Se toman solo esas 4 columnas del df
    df = df[['Fecha', 'Plazo real (días calendario)', 'IBR_efectivo', 'IBR_nominal']]
    df.rename(columns = {'Plazo real (días calendario)' : 'Plazo'}, inplace = True)
    # Se le hacen adecuaciones a la fecha para dejarla en formato YYYYMMDD numerico
    df['Fecha'] = pd.to_datetime(df['Fecha']).dt.strftime('%Y%m%d').astype('int32')
    df['Plazo'].fillna(0, inplace = True)
    df['Plazo'] = df['Plazo'].astype('int32')
    
    # Se rellena con 0 las tasas que esten en blanco o nulas
    df['IBR_efectivo'].fillna('0', inplace = True)
    df['IBR_nominal'].fillna('0', inplace = True)
    # Se convierten las tasas de tipo texto a decimal
    df.IBR_efectivo.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_efectivo'] = df['IBR_efectivo'].astype(float)
    df.IBR_nominal.replace({',':'.'}, regex = True, inplace = True)
    df['IBR_nominal'] = df['IBR_nominal'].astype(float)
    
    return df

In [9]:
def nominal_to_efectivo(tasa):
    '''Funcion para convertir tasa nominal a efectiva'''
    tasa = tasa / 100
    tasa_efectiva = (((1 + (tasa / 12))**12) - 1) * 100
    return tasa_efectiva

In [10]:
ibr_overnight = clean_ibr_df(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
ibr_overnight = ibr_overnight[ibr_overnight['Fecha'] >= date_2year_ago]
ibr_overnight

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20210714,1,1.752,1.713
1,20210713,1,1.753,1.714
2,20210712,1,1.751,1.712
3,20210709,3,1.751,1.712
4,20210708,1,1.753,1.714
...,...,...,...,...
856,20180109,1,4.723,4.552
857,20180105,4,4.713,4.543
858,20180104,1,4.714,4.543
859,20180103,1,4.689,4.520


In [11]:
ibr_mes = clean_ibr_df(r'C:\Users\Efra\Downloads\1.2.IBR_Plazo un mes nominal para un rango de fechas dado IQY.xlsx')
ibr_mes = ibr_mes[ibr_mes['Fecha'] >= date_2year_ago]
ibr_mes

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20210714,34,1.752,1.714
1,20210713,31,1.749,1.711
2,20210712,31,1.748,1.710
3,20210709,31,1.751,1.713
4,20210708,32,1.752,1.714
...,...,...,...,...
856,20180109,31,4.665,4.506
857,20180105,31,4.671,4.511
858,20180104,32,4.681,4.521
859,20180103,33,4.690,4.530


In [12]:
def interpolacion(x0, y0, x1, y1):
    y = []
    x = []
    for i in range(x0, x1):
        resul = y0 + ((y1-y0)/(x1-x0)) * (i-x0)
        y.append(resul)
        x.append(i)
    return x,y
    
    

In [13]:
x, y = interpolacion(1, 4.668, 31, 4.726)
print(x)
print(y)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[4.668, 4.669933333333334, 4.671866666666666, 4.6738, 4.6757333333333335, 4.677666666666667, 4.6796, 4.681533333333333, 4.683466666666667, 4.6854000000000005, 4.687333333333333, 4.689266666666667, 4.6912, 4.693133333333334, 4.6950666666666665, 4.697, 4.698933333333334, 4.700866666666666, 4.7028, 4.704733333333333, 4.706666666666667, 4.7086, 4.710533333333333, 4.712466666666667, 4.7144, 4.716333333333333, 4.718266666666667, 4.7202, 4.722133333333334, 4.724066666666666]


In [14]:
x, y = interpolacion(31, 4.726, 90, 4.65)
print(x)
print(y)

[31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
[4.726, 4.72471186440678, 4.723423728813559, 4.722135593220339, 4.720847457627118, 4.7195593220338985, 4.718271186440678, 4.716983050847458, 4.715694915254237, 4.714406779661017, 4.713118644067797, 4.711830508474576, 4.710542372881356, 4.709254237288135, 4.7079661016949155, 4.706677966101695, 4.705389830508475, 4.704101694915255, 4.702813559322034, 4.701525423728814, 4.700237288135593, 4.698949152542373, 4.697661016949152, 4.6963728813559324, 4.695084745762712, 4.693796610169492, 4.692508474576272, 4.691220338983051, 4.689932203389831, 4.68864406779661, 4.68735593220339, 4.686067796610169, 4.684779661016949, 4.683491525423729, 4.682203389830509, 4.680915254237289, 4.679627118644068, 4.678338983050848, 4.677050847457627, 4.675762711864407, 4.674474576271

In [15]:
x, y = interpolacion(90, 4.65, 182, 4.427)
print(x)
print(y)

[90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181]
[4.65, 4.647576086956522, 4.6451521739130435, 4.642728260869566, 4.6403043478260875, 4.637880434782609, 4.635456521739131, 4.633032608695652, 4.630608695652175, 4.628184782608696, 4.625760869565218, 4.623336956521739, 4.620913043478261, 4.618489130434783, 4.616065217391305, 4.613641304347826, 4.611217391304348, 4.6087934782608695, 4.606369565217392, 4.6039456521739135, 4.601521739130435, 4.599097826086957, 4.596673913043478, 4.594250000000001, 4.591826086956522, 4.589402173913044, 4.586978260869565, 4.584554347826087, 4.582130434782608, 4.5797

In [17]:
ibr_trimestral = clean_ibr_df(r'C:\Users\Efra\Downloads\1.3.IBR_Plazo tres meses nominal para un rango de fechas dado IQY.xlsx')
ibr_trimestral = ibr_trimestral[ibr_trimestral['Fecha'] >= date_2year_ago]
ibr_trimestral

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20210714,92,1.858,1.820
1,20210713,92,1.848,1.810
2,20210712,92,1.844,1.806
3,20210709,94,1.892,1.853
4,20210708,92,1.920,1.880
...,...,...,...,...
856,20180109,90,4.538,4.401
857,20180105,90,4.537,4.400
858,20180104,90,4.550,4.413
859,20180103,90,4.585,4.446


In [18]:
ibr_mes_nominal = clean_ibr_df(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx')
ibr_mes_nominal

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20120731,0,0.0,5.015
1,20120724,0,0.0,5.161
2,20120717,0,0.0,5.160
3,20120710,0,0.0,5.145
4,20120703,0,0.0,5.098
...,...,...,...,...
234,20080205,0,0.0,9.073
235,20080129,0,0.0,9.055
236,20080122,0,0.0,9.070
237,20080115,0,0.0,9.078


In [19]:
ibr_mes_nominal['IBR_efectivo'] = ibr_mes_nominal['IBR_nominal'].apply(nominal_to_efectivo)
ibr_mes_nominal

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20120731,0,5.131893,5.015
1,20120724,0,5.284849,5.161
2,20120717,0,5.283800,5.160
3,20120710,0,5.268076,5.145
4,20120703,0,5.218822,5.098
...,...,...,...,...
234,20080205,0,9.459970,9.073
235,20080129,0,9.440416,9.055
236,20080122,0,9.456710,9.070
237,20080115,0,9.465402,9.078


In [20]:
ibr_semestral = clean_ibr_df(r'C:\Users\Efra\Downloads\1.5.IBR_Plazo seis meses nominal para un rango de fechas dado IQY.xlsx')
ibr_semestral = ibr_semestral[ibr_semestral['Fecha'] >= date_2year_ago]
ibr_semestral

,Fecha,Plazo,IBR_efectivo,IBR_nominal
0,20210714,184,2.266,2.223
1,20210713,184,2.251,2.208
2,20210712,184,2.222,2.180
3,20210709,186,2.265,2.222
4,20210708,187,2.319,2.274
...,...,...,...,...
856,20180109,181,4.382,4.275
857,20180105,181,4.366,4.260
858,20180104,181,4.374,4.268
859,20180103,181,4.388,4.281


In [21]:
ibr_interpolacion1 = ibr_overnight.merge(ibr_mes, on='Fecha', suffixes=('_on', '_1m'))
ibr_interpolacion2 = ibr_trimestral.merge(ibr_semestral, on='Fecha', suffixes=('_3m', '_6m'))
ibr_interpolacion = ibr_interpolacion1.merge(ibr_interpolacion2,  on='Fecha')

In [22]:
ibr_interpolacion

,Fecha,Plazo_on,IBR_efectivo_on,IBR_nominal_on,Plazo_1m,IBR_efectivo_1m,IBR_nominal_1m,Plazo_3m,IBR_efectivo_3m,IBR_nominal_3m,Plazo_6m,IBR_efectivo_6m,IBR_nominal_6m
0,20210714,1,1.752,1.713,34,1.752,1.714,92,1.858,1.820,184,2.266,2.223
1,20210713,1,1.753,1.714,31,1.749,1.711,92,1.848,1.810,184,2.251,2.208
2,20210712,1,1.751,1.712,31,1.748,1.710,92,1.844,1.806,184,2.222,2.180
3,20210709,3,1.751,1.712,31,1.751,1.713,94,1.892,1.853,186,2.265,2.222
4,20210708,1,1.753,1.714,32,1.752,1.714,92,1.920,1.880,187,2.319,2.274
...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,20180109,1,4.723,4.552,31,4.665,4.506,90,4.538,4.401,181,4.382,4.275
857,20180105,4,4.713,4.543,31,4.671,4.511,90,4.537,4.400,181,4.366,4.260
858,20180104,1,4.714,4.543,32,4.681,4.521,90,4.550,4.413,181,4.374,4.268
859,20180103,1,4.689,4.520,33,4.690,4.530,90,4.585,4.446,181,4.388,4.281


In [69]:
def generar_df_interpolado(f_ini, f_fin):
    
    fecha_ini = str(f_ini)
    fecha_fin = str(f_fin)
    sdate = date(int(fecha_ini[0:4]), int(fecha_ini[4:6]), int(fecha_ini[6:8]))   # start date
    edate = date(int(fecha_fin[0:4]), int(fecha_fin[4:6]), int(fecha_fin[6:8]))   # end date

    delta = edate - sdate       # as timedelta
    df_resul = pd.DataFrame(columns = ['fecha'])
    list_res = []
    flag = 0
    for i in range(delta.days + 1):
        day = sdate + timedelta(days=i)
        print(day)
        df = pd.DataFrame(columns = ['fecha'])
        fecha = int(day.strftime('%Y%m%d'))
        ibr = ibr_interpolacion[ibr_interpolacion['Fecha'] == fecha]
        #print(ibr.values)
        x_c1,y_c1 = interpolacion(x0=1, y0=ibr.iloc[0]['IBR_efectivo_on'], 
        x1=31, y1=ibr.iloc[0]['IBR_efectivo_1m'])
        x_c2,y_c2 = interpolacion(x0=31, y0=ibr.iloc[0]['IBR_efectivo_1m'], 
        x1=91, y1=ibr.iloc[0]['IBR_efectivo_3m'])
        x_c3,y_c3 = interpolacion(x0=91, y0=ibr.iloc[0]['IBR_efectivo_3m'], 
        x1=182, y1=ibr.iloc[0]['IBR_efectivo_6m'])
        if flag == 0:
            days = x_c1 + x_c2 + x_c3
            flag+=1
        tasas = y_c1 + y_c2 + y_c3
        #print(days)
        df = df.reindex(columns = df.columns.tolist() + days)
        data = [fecha] + tasas
        df.loc[len(df)] = data
        list_res.append(df)

    df_resul = pd.concat(list_res)
    return df_resul

In [24]:
from datetime import date, timedelta, datetime
fecha_ini = str(20080815)
fecha_fin = str(20080915)
sdate = date(int(fecha_ini[0:4]), int(fecha_ini[4:6]), int(fecha_ini[6:8]))   # start date
edate = date(int(fecha_fin[0:4]), int(fecha_fin[4:6]), int(fecha_fin[6:8]))   # end date

delta = edate - sdate       # as timedelta

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    print(day)
    print(int(day.strftime('%Y%m%d')))


2008-08-15
20080815
2008-08-16
20080816
2008-08-17
20080817
2008-08-18
20080818
2008-08-19
20080819
2008-08-20
20080820
2008-08-21
20080821
2008-08-22
20080822
2008-08-23
20080823
2008-08-24
20080824
2008-08-25
20080825
2008-08-26
20080826
2008-08-27
20080827
2008-08-28
20080828
2008-08-29
20080829
2008-08-30
20080830
2008-08-31
20080831
2008-09-01
20080901
2008-09-02
20080902
2008-09-03
20080903
2008-09-04
20080904
2008-09-05
20080905
2008-09-06
20080906
2008-09-07
20080907
2008-09-08
20080908
2008-09-09
20080909
2008-09-10
20080910
2008-09-11
20080911
2008-09-12
20080912
2008-09-13
20080913
2008-09-14
20080914
2008-09-15
20080915


In [75]:
df = generar_df_interpolado(20180102, 20180106)
df['fecha'] = df['fecha'].astype('int32')
df
#ibr_interpolacion.iloc[0]['IBR_efectivo_on']

2018-01-02
2018-01-03
2018-01-04
2018-01-05
2018-01-06


IndexError: single positional indexer is out-of-bounds

In [38]:
df.values

array([], shape=(0, 0), dtype=float64)

# Tratamiento archivos Ice Libor

In [26]:
# Leemos la ruta donde se encuentra el proyecto
path = os.getcwd()
# Le anadimos la parte de los insumos
dir_insumos = path + '\Insumos\libor\\'
dir_insumos

'D:\\Proyectos\\proyecto_hector\\Insumos\\libor\\'

In [27]:
# Leemos todos los archivos en la carpeta
files = os.listdir(dir_insumos)
# Recorremos la carpeta para sacar solo los que tengan extension csv
files_csv = [f for f in files if f[-3:] == 'csv']
# Inicializamos una variable tipo dataframe
df_libor = pd.DataFrame()

# Recorrermos la lista de archivos uno a uno
for files in files_csv:
    path = dir_insumos + files
    # Se lee cada csv con la funcion read y se concatena para dejarlo en un solo df
    data = pd.read_csv(path, delimiter = ',')
    df_libor = df_libor.append(data)
    
df_libor = df_libor[['Benchmark Date', 'Tenor', 'USD ICE LIBOR']]

In [28]:
df_libor_pivot = pd.pivot_table(df_libor, values='USD ICE LIBOR', index='Benchmark Date',
                    columns=['Tenor'], aggfunc=np.sum)

In [29]:
df_libor

,Benchmark Date,Tenor,USD ICE LIBOR
0,"January 2, 2019",Overnight,2.38675
1,"January 2, 2019",1 Week,2.41475
2,"January 2, 2019",1 Month,2.50713
3,"January 2, 2019",2 Month,2.61738
4,"January 2, 2019",3 Month,2.79388
...,...,...,...
93,"April 22, 2021",1 Month,0.10613
94,"April 22, 2021",2 Month,0.15013
95,"April 22, 2021",3 Month,0.17575
96,"April 22, 2021",6 Month,0.21063


In [30]:
df_libor_pivot

Tenor,1 Month,1 Week,1 Year,2 Month,3 Month,6 Month,Overnight
Benchmark Date,,,,,,,
"April 1, 2019",2.49338,2.42125,2.73000,2.56188,2.59550,2.66913,2.38813
"April 1, 2020",1.01625,0.42863,1.00238,1.24450,1.43650,1.19525,0.10875
"April 1, 2021",0.11038,0.08488,0.28050,0.13525,0.19975,0.20125,0.07475
"April 10, 2019",2.48200,2.41425,2.74013,2.54850,2.60350,2.62700,2.38850
"April 11, 2019",2.47263,2.40775,2.73413,2.53675,2.59675,2.63125,2.38850
...,...,...,...,...,...,...,...
"September 6, 2019",2.04900,2.13150,1.94900,2.14450,2.13413,2.03413,2.09475
"September 7, 2020",0.15563,0.10675,0.42100,0.18325,0.24175,0.29325,NaN
"September 8, 2020",0.15550,0.10875,0.42700,0.20413,0.24950,0.30100,0.07938


In [31]:
'''try:
    print('Borrando archivos IBR...')
    os.remove(r'C:\Users\Efra\Downloads\1.1.IBR_Plazo overnight nominal para un rango de fechas dado IQY.xlsx')
    os.remove(r'C:\Users\Efra\Downloads\1.2.IBR_Plazo un mes nominal para un rango de fechas dado IQY.xlsx')
    os.remove(r'C:\Users\Efra\Downloads\1.3.IBR_Plazo tres meses nominal para un rango de fechas dado IQY.xlsx')
    os.remove(r'C:\Users\Efra\Downloads\1.4.IBR_Plazo mes vencido nominal para un rango de fechas dado IQY.xlsx')
    os.remove(r'C:\Users\Efra\Downloads\1.5.IBR_Plazo seis meses nominal para un rango de fechas dado IQY.xlsx')
    print('Archivos borrados')
except :
    print('Todos los archivos han sido borrados anteriormente')'''

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 61-62: truncated \UXXXXXXXX escape (<ipython-input-31-85866e4365b1>, line 10)